In [1]:
import torch
from torch.nn import BCELoss
import torch.nn as nn
from pprint import pformat

from importlib import reload

import vitmbt
import mbt
import ablation
import helpers
reload(mbt)
reload(ablation)
reload(helpers)
reload(vitmbt)
from helpers import ClassifierMetrics
from vitmbt import ViTMBT, train, val
from data import load_data
import warnings
warnings.filterwarnings('ignore')

from constants import *
from loss import multicrop_loss

import torch.optim.lr_scheduler as lrsch

DEVICE = "cuda"
RESULTS = "results/best.txt"
PRETRAINED_CHKPT = "./pretrained_models/L_16-i21k-300ep-lr_0.001-aug_medium1-wd_0.1-do_0.1-sd_0.1--imagenet2012-steps_20k-lr_0.01-res_224.npz"
WARMUP_EPOCHS = 5
EPOCHS = WARMUP_EPOCHS + 30
lr = 0.00005
betas = (0.9, 0.999)
momentum = 0.9
BATCH_SZ = 2
LABELS = 8
SPLIT = [0.95, 0.05, 0.0]
SPLIT = [1, 0.0, 0.0]
MILESTONES = [WARMUP_EPOCHS]
T_0 = 6
PT_ATTN_DROPOUT = 0.15
ATTN_DROPOUT = 0.4
LINEAR_DROPOUT = 0.1
BOTTLE_LAYER = 10
FREEZE_FIRST = 8
TOTAL_LAYERS = 14
APPLY_AUG = True

train_dl, val_dl, test_dl  = load_data(f"{DATA_DIR}/Labels/all_pruned.csv", 
                                       batch_sz=BATCH_SZ,
                                       train_val_test_split=SPLIT, nlines=4)

In [2]:

mbt_teacher = ViTMBT(1024, num_class=LABELS, no_class=False, bottle_layer=BOTTLE_LAYER, freeze_first=FREEZE_FIRST, num_layers=TOTAL_LAYERS, apply_augmentation=APPLY_AUG, attn_drop=ATTN_DROPOUT, linear_drop=LINEAR_DROPOUT)
mbt_teacher = nn.DataParallel(mbt_teacher).cuda()

mbt_student = ViTMBT(1024, num_class=LABELS, no_class=False, bottle_layer=BOTTLE_LAYER, freeze_first=FREEZE_FIRST, num_layers=TOTAL_LAYERS, apply_augmentation=APPLY_AUG, attn_drop=ATTN_DROPOUT, linear_drop=LINEAR_DROPOUT)
mbt_student = nn.DataParallel(mbt_student).cuda()




# only the student's weights are updated by the optimiser
optimizer = torch.optim.AdamW(mbt_student.parameters(), betas=(0.9, 0.999), lr=lr, weight_decay=0.4)

scheduler = lrsch.SequentialLR(
    optimizer=optimizer,
    schedulers=[
    lrsch.ConstantLR(optimizer=optimizer, factor=1, total_iters=WARMUP_EPOCHS),
    lrsch.CosineAnnealingWarmRestarts(optimizer=optimizer, T_0=T_0)
], milestones=MILESTONES)



train_cls = ClassifierMetrics(task='multilabel', n_labels=LABELS, device=DEVICE)
val_cls = ClassifierMetrics(task='multilabel', n_labels=LABELS, device=DEVICE)


Loading audio layers...
torch.Size([1, 1, 1024]) torch.Size([1, 184, 1024])
Loaded successfully in 3.8s
Loading video layers...
Loaded successfully in 3.8s
Reset trainable parameters of layer = 8.norm1
Reset trainable parameters of layer = 8.attn.qkv
Reset trainable parameters of layer = 8.attn.proj
Reset trainable parameters of layer = 8.norm2
Reset trainable parameters of layer = 8.mlp.fc1
Reset trainable parameters of layer = 8.mlp.fc2
Reset trainable parameters of layer = 9.norm1
Reset trainable parameters of layer = 9.attn.qkv
Reset trainable parameters of layer = 9.attn.proj
Reset trainable parameters of layer = 9.norm2
Reset trainable parameters of layer = 9.mlp.fc1
Reset trainable parameters of layer = 9.mlp.fc2
Reset trainable parameters of layer = 8.norm1
Reset trainable parameters of layer = 8.attn.qkv
Reset trainable parameters of layer = 8.attn.proj
Reset trainable parameters of layer = 8.norm2
Reset trainable parameters of layer = 8.mlp.fc1
Reset trainable parameters of l

In [3]:
for epoch in range(EPOCHS):
    train_loss = train(mbt_teacher, mbt_student, train_dl, optimizer, CENTRE_CONSTANT, loss_fn=multicrop_loss)
    val_loss = val(mbt_teacher, mbt_student, val_dl, CENTRE_CONSTANT, loss_fn=multicrop_loss)
    scheduler.step()

    print(
        (f"Epoch {epoch + 1}: train_loss {train_loss:.5f}, val_loss {val_loss:.5f}\n")
        )


  0%|          | 0/2 [00:00<?, ?it/s]

TypeError: cannot unpack non-iterable float object